In [9]:
import os
import sys
import mne
import time
import pickle
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.preprocessing import ICA, create_eog_epochs
from scipy.stats import skew, kurtosis
from scipy.signal import argrelextrema
import heartpy as hp
import pprint

In [4]:
root_folder = 'C:/Users/robbi/Desktop/BMED_6517_emotional_state_classifier/Data/DEAP'
data_folter = root_folder + '/data_original'
ratings_csv_path = root_folder + '/Metadata/metadata_csv/participant_ratings.csv'
DEAP_EEG_channels_xlsx_path = root_folder + '/Metadata/DEAP_EEG_channels.xlsx'

In [10]:
N_trials = 40
N_EEG_electrodes = 32
N_BIPOLAR = 16
sfreq = 128

In [11]:
npy_folder = os.path.join(os.getcwd(), 'results', 'npy')

In [12]:
for folder in [npy_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)

In [13]:
def epoch_bdf(subject_id):
    npy_plet_path = os.path.join(npy_folder, 's{:02}_plet.npy'.format(subject_id))
    npy_GSR_path = os.path.join(npy_folder, 's{:02}_GSR.npy'.format(subject_id))

    print('\n----------------------------------------------------\n')
    print('extracting bipolar data for Subject {:02}'.format(subject_id))

    bdf_file_name = 's{:02d}.bdf'.format(subject_id)
    bdf_file_path = os.path.join(data_folter, bdf_file_name)

    print('Loading .bdf file {}'.format(bdf_file_path))
    raw = mne.io.read_raw_bdf(bdf_file_path, preload=True, verbose=False).load_data()

    ch_names = raw.ch_names
    eeg_channels = ch_names[:N_EEG_electrodes]
    non_eeg_channels = ch_names[N_EEG_electrodes:]
    stim_ch_name = ch_names[-1]
    stim_channels = [ stim_ch_name ]

    raw_copy = raw.copy()
    raw_stim = raw_copy.pick_channels(stim_channels)
    raw.pick_channels(non_eeg_channels)
    print("Done!")

    #####################################################
    print('Getting events from the status channel')
    events = mne.find_events(raw_stim, stim_channel=stim_ch_name, verbose=True)
    if subject_id<=23:
        # Subject 1-22 and Subjects 23-28 have 48 channels.
        # Subjects 29-32 have 49 channels.
        # For Subjects 1-22 and Subject 23, the stimuli channel has the name 'Status'
        # For Subjects 24-28, the stimuli channel has the name ''
        # For Subjects 29-32, the stimuli channels have the names '-0' and '-1'
        pass
    else:
        # The values of the stimuli channel have to be changed for Subjects 24-32
        # Trigger channel has a non-zero initial value of 1703680 (consider using initial_event=True to detect this event)
        events[:,2] -= 1703680 # subtracting initial value
        events[:,2] = events[:,2] % 65536 # getting modulo with 65536

    print('')
    event_IDs = np.unique(events[:,2])
    for event_id in event_IDs:
        col = events[:,2]
        print('Event ID {} : {:05}'.format(event_id, np.sum( 1.0*(col==event_id) ) ) )
    inds_new_trial = np.where(events[:,2] == 4)[0]
    events_new_trial = events[inds_new_trial,:]
    baseline = (0, 0)
    print('Epoching the data, into [-5sec, +60sec] epochs')
    epochs = mne.Epochs(raw, events_new_trial, event_id=4, tmin=-5.0, tmax=60.0, picks=non_eeg_channels, baseline=baseline, preload=True)

    
    epochs_copy = epochs.copy()
    print('Downsampling epoched data to 128Hz')
    epochs_downsampled = epochs_copy.resample(sfreq)

    data = epochs.get_data()
    data_downsampled = epochs_downsampled.get_data()
    print('Original epoched data shape: {}'.format(data.shape))
    print('Downsampled epoched data shape: {}'.format(data_downsampled.shape))

    #############################

    ratings = pd.read_csv(ratings_csv_path)
    is_subject =  (ratings['Participant_id'] == subject_id)
    ratings_subj = ratings[is_subject]
    trial_pick_indices = []
    print('\nPreparing EEG trial reordering, from presentation order, to video (Experiment_id) order')
    for i in range(N_trials):
        exp_id = i+1
        is_exp = (ratings['Experiment_id'] == exp_id)
        trial_id = ratings_subj[is_exp]['Trial'].values[0]
        trial_pick_indices.append(trial_id - 1)
        print('Picking source (raw) trial #{:02} to fill target (npy) trial #{:02} | Experiment_id: {:02}'.format(trial_id, exp_id, exp_id))

    #############################    

    # Store clean and reordered data to numpy array
    epoch_duration = data_downsampled.shape[-1]
    data_npy = np.zeros((N_trials, N_BIPOLAR, epoch_duration))
    print(data_npy.shape)
    print('\nStoring the final EEG data in a numpy array of shape {}'.format(data_npy.shape))
    print(len(trial_pick_indices))
    for trial_source, trial_target in zip(trial_pick_indices, range(N_trials)):
        data_trial = data_downsampled[trial_source]
        print(data_trial.shape)
        print(data_npy[trial_target,:,:].shape)
        data_npy[trial_target,:,:] = data_trial.copy()
    print('Saving the final EEG data in a .npy file')
    np.save(npy_plet_path, data_npy[:,13,:])
    np.save(npy_GSR_path, data_npy[:,8,:])

    print('Raw physiological data has been epoched, downsampled, trial-reordered and channel-reordered.')
    print('Finished.')    

In [14]:
def extract_HR_features(subject_id,trial_num):

    trial = trial_num

    feats_folder = os.path.join(os.getcwd(), 'results', 'features_new')

    plet_path = os.path.join(feats_folder, 's{:02}_plet.npy'.format(subject_id))
    if os.path.exists(plet_path):
        print('\nFeatures are already saved.\nSkipping data extraction for Subject {:02}'.format(subject_id))

    print('\nExtracting features for Subject {:02}'.format(subject_id))

    # Load data
    plet_file_path = os.path.join(npy_folder, 's{:02d}_plet.npy'.format(subject_id))
    print('Loading preprocessed Plethysmograph from .npy file {}\n'.format(plet_path))
    plet_data = np.load(plet_file_path)

    wd, m = hp.process(plet_data[trial,:], sample_rate = sfreq)

    ## set large figure
    # plt.figure(figsize=(12,4))

    ## call plotter
    # hp.plotter(wd, m)
    # plt.show()
    
    return m

In [15]:
def statistics_feature(data):
    mean = np.mean(data)
    std = np.std(data)
    Skew = skew(data)
    kurt = kurtosis(data)
    mean_fst_absdiff = np.mean(abs(np.diff(data)))
    mean_snd_absdiff = np.mean(abs(np.diff(np.diff(data))))
    mean_fst_diff = np.mean(np.diff(data))
    mean_snd_diff = np.mean(np.diff(np.diff(data)))
    mean_neg_diff = np.mean(np.diff(data)[np.where(np.diff(data)<0)])
    proportion_neg_diff = len(np.where(np.diff(data)<0)[0])/(len(np.diff(data)))
    number_local_min = len(argrelextrema(data,np.less)[0])
    number_local_max = len(argrelextrema(data,np.greater)[0])
    f1 = [mean,std,Skew,kurt]
    f2 = [mean_fst_absdiff,mean_snd_absdiff,mean_fst_diff,mean_snd_diff,mean_neg_diff,proportion_neg_diff]
    f3 = [number_local_min,number_local_max]
    f = f1+f2+f3
    names = ['mean','std','skew','kurtosis','me.1absdf','me.2absdf','me.1df','me.2df','me.negdf','ro.negdf','num.argmi','num.argma']
    GSR_features = dict(zip(names, f))
    return GSR_features

In [16]:
def extract_GSR_features(subject_id,trial_num):

    trial = trial_num

    feats_folder = os.path.join(os.getcwd(), 'results', 'features_new')

    GSR_path = os.path.join(feats_folder, 's{:02}_GSR.npy'.format(subject_id))
    if os.path.exists(GSR_path):
        print('\nFeatures are already saved.\nSkipping data extraction for Subject {:02}'.format(subject_id))

    print('\nExtracting features for Subject {:02}'.format(subject_id))

    # Load data
    GSR_file_path = os.path.join(npy_folder, 's{:02d}_GSR.npy'.format(subject_id))
    print('Loading preprocessed GSR from .npy file {}\n'.format(GSR_path))
    GSR_data = np.load(GSR_file_path)

    # plt.plot(GSR_data[trial,:])
    # plt.show()
    
    features = statistics_feature(GSR_data[trial,:])
    return features


In [24]:
trial_num = 1

HR_feat = extract_HR_features(subject_id, trial_num)
for measure in HR_feat.keys():
    print('%s: %f' %(measure, HR_feat[measure]))


Extracting features for Subject 01
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_plet.npy

bpm: 59.155681
ibi: 1014.272837
sdnn: 71.712838
sdsd: 58.072320
rmssd: 80.236414
pnn20: 0.695652
pnn50: 0.391304
hr_mad: 39.062500
sd1: 56.694517
sd2: 72.648937
s: 12939.580440
sd1/sd2: 0.780390
breathingrate: 0.233333


In [25]:
gsr_feat = extract_GSR_features(subject_id, trial_num)
#pprint.pprint(gsr_feat)

for measure in gsr_feat.keys():
    print('%s: ' %(measure),gsr_feat[measure])


Extracting features for Subject 01
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_GSR.npy

mean:  33.10075522876701
std:  40.565825590445066
skew:  1.0164400576522343
kurtosis:  0.0742482448745676
me.1absdf:  0.12912574087394774
me.2absdf:  0.17238428098780084
me.1df:  0.008228656585115995
me.2df:  3.3211131007407116e-05
me.negdf:  -0.10948648423675061
ro.negdf:  0.5521096285611251
num.argmi:  2420
num.argma:  2419


In [26]:
with open('C:\\Users\\robbi\\Desktop\\BMED_6517_emotional_state_classifier\\Data\\DEAP\\data_preprocessed_python\\s01.dat','rb') as file:
    subj_labels = pickle.load(file,encoding='latin1')['labels']

In [27]:
print(subj_labels[:15,:])
print(len(HR_feat))
print(len(gsr_feat))
subj_array = np.zeros([40,len(HR_feat)+len(gsr_feat)])
subj_array.shape

[[7.71 7.6  6.9  7.83]
 [8.1  7.31 7.28 8.47]
 [8.58 7.54 9.   7.08]
 [4.94 6.01 6.12 8.06]
 [6.96 3.92 7.19 6.05]
 [8.27 3.92 7.   8.03]
 [7.44 3.73 7.08 7.04]
 [7.32 2.55 6.32 5.87]
 [4.04 3.29 3.62 5.99]
 [1.99 4.86 2.04 7.09]
 [2.99 2.36 3.63 6.24]
 [2.71 2.77 3.4  7.35]
 [1.95 3.12 2.87 6.18]
 [4.18 2.24 3.04 5.04]
 [3.17 8.08 2.91 5.04]]
13
12


(40, 25)

In [29]:
# extract gsr, and hr data for all subjects
full_data = np.zeros([40*28,len(HR_feat)+len(gsr_feat)+3])
columns = np.empty([0])
k = 0 
bad_signals_hr = [13,22]
bad_signal = [29]
for subject in range(1,29):
    if subject not in bad_signal:
        epoch_bdf(subject)
    with open('C:\\Users\\robbi\\Desktop\\BMED_6517_emotional_state_classifier\\Data\\DEAP\\data_preprocessed_python\\s{:02d}.dat'.format(subject),'rb') as file:
        subj_labels = pickle.load(file,encoding='latin1')['labels']
        
    subj_array = np.zeros([40,len(HR_feat)+len(gsr_feat)-2])
    for trial in range(40):
        try:
            HR_feat = extract_HR_features(subject, trial)
            hr_bad_sig = False
        except:
            hr_bad_sig = True
        try:
            gsr_feat = extract_GSR_features(subject, trial)
            gsr_bad_sig = False
        except:
            gsr_bad_sig = True
        i = 0
        for metric in HR_feat.keys():
            if not hr_bad_sig:
                subj_array[trial-1,i] = HR_feat[metric]
            else:
                subj_array[trial-1,i] = 0
            if k == 0:
                columns = np.append(columns,metric)
            i+=1
        for metric in gsr_feat.keys():
            if (metric != 'skew') and (metric != 'kurtosis'):
                if not gsr_bad_sig:
                    subj_array[trial-1,i] = gsr_feat[metric]
                else:
                    subj_array[trial-1,i] = 0
                if k == 0:   
                    columns = np.append(columns,metric)
                i+=1  
        k = 1
    if len(columns)<27:
        columns = np.append(columns,['valence','arousal','dominance','liking'])
    subj = np.concatenate((subj_array,subj_labels),axis=1)
    subj_1 = np.concatenate((columns.reshape(1,len(columns)),subj), axis=0)
    
    # save individual results
    np.save(npy_folder+'\s{:02d}.npy'.format(subject),subj_1)
    full_data[(subject-1)*40:subject*40,1:] = subj_1[1:,:]
id = np.empty([0])
for i in range(1,29):
    id=np.append(id,np.ones([40,1])*i)
id.reshape([len(id),1])
full_data[:,0] = id
columns = np.concatenate((['subject'],columns))
full_data = np.concatenate((columns.reshape(1,len(columns)),full_data),axis=0)
np.save(npy_folder+'\s_all.npy',full_data)


----------------------------------------------------

extracting bipolar data for Subject 01
Loading .bdf file C:/Users/robbi/Desktop/BMED_6517_emotional_state_classifier/Data/DEAP/data_original\s01.bdf
Done!
Getting events from the status channel
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
13760 events found
Event IDs: [1 2 3 4 5 6 7]

Event ID 1 : 162.0
Event ID 2 : 002.0
Event ID 3 : 040.0
Event ID 4 : 040.0
Event ID 5 : 040.0
Event ID 6 : 13475.0
Event ID 7 : 001.0
Epoching the data, into [-5sec, +60sec] epochs
Not setting metadata
40 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 40 events and 33281 original time points ...
0 bad epochs dropped
Downsampling epoched data to 128Hz
Original epoched data shape: (40, 16, 33281)
Downsampled epoched data shape: (40, 16, 8320)

Preparing E

C:\Users\robbi\AppData\Local\Temp\ipykernel_26284\801245321.py:72: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  trial_id = ratings_subj[is_exp]['Trial'].values[0]


(40, 23)

Extracting features for Subject 01
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_plet.npy


Extracting features for Subject 01
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_GSR.npy


Extracting features for Subject 01
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_plet.npy


Extracting features for Subject 01
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_GSR.npy


Extracting features for Subject 01
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_plet.npy


Extracting features for Subject 01
Loading preprocessed GSR from

C:\Users\robbi\anaconda3\envs\deep\lib\site-packages\scipy\interpolate\_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)



Extracting features for Subject 01
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_GSR.npy


Extracting features for Subject 01
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_plet.npy


Extracting features for Subject 01
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_GSR.npy


Extracting features for Subject 01
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_plet.npy


Extracting features for Subject 01
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_GSR.npy


Extracting features for Subject 01
Loading preprocessed Plethysmograph from .npy file

...

(40, 23)

Extracting features for Subject 27
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_plet.npy


Extracting features for Subject 27
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_GSR.npy


Extracting features for Subject 27
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_plet.npy


Extracting features for Subject 27
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_GSR.npy


Extracting features for Subject 27
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_plet.npy


Extracting features for Subject 27
Loading preprocessed GSR from

C:\Users\robbi\anaconda3\envs\deep\lib\site-packages\scipy\interpolate\_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)



Extracting features for Subject 27
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_GSR.npy


Extracting features for Subject 27
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_plet.npy


Extracting features for Subject 27
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_GSR.npy


Extracting features for Subject 27
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_plet.npy


Extracting features for Subject 27
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s27_GSR.npy


Extracting features for Subject 27
Loading preprocessed Plethysmograph from .npy file

C:\Users\robbi\AppData\Local\Temp\ipykernel_26284\801245321.py:72: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  trial_id = ratings_subj[is_exp]['Trial'].values[0]


(40, 23)

Extracting features for Subject 28
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s28_plet.npy


Extracting features for Subject 28
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s28_GSR.npy


Extracting features for Subject 28
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s28_plet.npy


Extracting features for Subject 28
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s28_GSR.npy


Extracting features for Subject 28
Loading preprocessed Plethysmograph from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s28_plet.npy


Extracting features for Subject 28
Loading preprocessed GSR from

In [178]:
id = np.empty([0])
for i in range(1,29):
    id=np.append(id,np.ones([40,1])*i)
id.reshape([len(id),1])
print(id[:,np.newaxis].shape)

(1120, 1)


In [180]:
columns = np.concatenate((['subject'],columns))
full_data = np.concatenate((columns.reshape(1,len(columns)),full_data),axis=0)
np.save(npy_folder+'\s_all.npy',full_data)

In [20]:
#test = np.load('C:\\Users\\robbi\\Desktop\\BMED_6517_emotional_state_classifier\\Scripts\\results\\npy\\s01_GSR.npy')
print(test.shape)
print(test[:5,:])
gsr_feat = extract_GSR_features(1, 4)
print(gsr_feat)

(40, 8320)
[[ 2.10432620e+01  2.10831970e+01  2.10391942e+01 ...  1.47142982e+01
   1.47178717e+01  1.47215275e+01]
 [ 2.28871833e+01  2.26773343e+01  2.25636245e+01 ...  9.14290984e+01
   9.12749763e+01  9.13413775e+01]
 [-1.66444565e+01 -1.66048150e+01 -1.66462991e+01 ...  5.76520147e+02
   5.76421781e+02  5.76284978e+02]
 [ 5.71854513e-01  4.70419441e-01  4.61145788e-01 ... -1.68169783e+02
  -1.68198881e+02 -1.68178665e+02]
 [ 3.61389845e+01  3.61103749e+01  3.61398552e+01 ... -2.54175478e+02
  -2.54190841e+02 -2.54331615e+02]]

Extracting features for Subject 01
Loading preprocessed GSR from .npy file C:\Users\robbi\Desktop\BMED_6517_emotional_state_classifier\Scripts\results\features_new\s01_GSR.npy

{'mean': -66.17489706814423, 'std': 106.64381789835507, 'skew': -0.005794149246370585, 'kurtosis': -1.043701781843589, 'me.1absdf': 0.10258941335614127, 'me.2absdf': 0.15318206995529665, 'me.1df': -0.03491652840936669, 'me.2df': -1.3484619633502382e-05, 'me.negdf': -0.1162512123523639